# Analyzing replicability of functional connectivity-based multivariate BWAS on the Human Connectome Project dataset

## Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.model_selection import KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from joblib import Parallel, delayed
from mlxtend.evaluate import permutation_test
sns.set(rc={"figure.figsize":(4, 2)})
sns.set_style("whitegrid")

## Load HCP data

We load functional network matrices (netmats) from the HCP1200-release, as published on connectomeDB: https://db.humanconnectome.org/
Due to licensoing issues, data is not supplied with the repository, but can be downloaded from the ConnectomeDB.
See [hcp_data/readme.md](hcp_data/readme.md) for more details.

In [2]:
# HCP data can be obtained from the connectomeDB
# data is not part of this repository
subjectIDs = pd.read_csv('hcp_data/subjectIDs.txt', header=None)

netmats_pearson = pd.read_csv('hcp_data/netmats1_correlationZ.txt',
                             sep=' ',
                             header=None)
netmats_pearson['ID'] = subjectIDs[0]
netmats_pearson.set_index('ID', drop=True, inplace=True)


netmats_parcor = pd.read_csv('hcp_data/netmats2_partial-correlation.txt',
                             sep=' ',
                             header=None)
netmats_parcor['ID'] = subjectIDs[0]
netmats_parcor.set_index('ID', drop=True, inplace=True)

behavior = pd.read_csv('hcp_data/hcp1200_behavioral_data.csv')
behavior = behavior.set_index('Subject', drop=True)

# convert age to numeric
age = []
for s in behavior['Age']:
    if s == '36+':
        age.append(36)
    else:
        split = s.split(sep='-')
        age.append(np.mean((float(split[0]), float(split[1]))))

behavior['age'] = age
behavior

,Release,Acquisition,Gender,Age,3T_Full_MR_Compl,T1_Count,T2_Count,3T_RS-fMRI_Count,3T_RS-fMRI_PctCompl,3T_Full_Task_fMRI,...,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final,age
Subject,,,,,,,,,,,,,,,,,,,,,
100004,S900,Q06,M,22-25,False,0,0,0,0.0,False,...,101.12,86.45,2.0,45.9,107.17,105.31,1.80,0.0,1.80,23.5
100206,S900,Q11,M,26-30,True,1,1,4,100.0,True,...,108.79,97.19,1.0,49.7,72.63,72.03,1.84,0.0,1.84,28.0
100307,Q1,Q01,F,26-30,True,1,1,4,100.0,True,...,101.12,86.45,0.0,38.6,71.69,71.76,1.76,0.0,1.76,28.0
100408,Q3,Q03,M,31-35,True,1,1,4,100.0,True,...,108.79,98.04,2.0,52.6,114.01,113.59,1.76,2.0,1.68,33.0
100610,S900,Q08,M,26-30,True,2,1,4,100.0,True,...,122.25,110.45,0.0,38.6,84.84,85.31,1.92,1.0,1.88,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992774,Q2,Q02,M,31-35,True,2,2,4,100.0,True,...,122.25,111.41,4.0,50.1,107.17,103.55,1.76,0.0,1.76,33.0
993675,S900,Q09,F,26-30,True,2,2,4,100.0,True,...,122.25,110.45,0.0,38.6,84.07,84.25,1.80,1.0,1.76,28.0
994273,S500,Q06,M,26-30,True,1,1,4,100.0,True,...,122.25,111.41,7.0,63.8,110.65,109.73,1.80,1.0,1.76,28.0


# Function to prepare target variable


In [3]:
def create_data(target='CogTotalComp_AgeAdj', feature_data=netmats_parcor):
    # it's a good practice to use pandas for merging, messing up subject order can be painful
    features = feature_data.columns
    df = behavior
    df = df.merge(feature_data, left_index=True, right_index=True, how='left')

    df = df.dropna(subset = [target] + features.values.tolist())
    y = df[target].values
    X = df[features].values
    return X, y

# Function implementing a single bootstrap iteration

We define a workhorse function which:
- randomly samples the discovery and the replication datasets,
- creates cross-validated estimates of predictive performance within the discovery sample
- finalizes the model by fitting it to the whole discovery sample (overfits the discovery but not the replication sample)
- use it to predict the replication sample

In [4]:
def bootstrap_workhorse(X, y, sample_size, model, random_state, shuffle_y=False):

    #create discovery and replication samples by random sampling from the whole dataset (without replacement)

    # if shuffle_y is true, a null model is created bz permuting y
    if shuffle_y:
        rng = np.random.default_rng(random_state)
        y = rng.permutation(y)

    # sample the discovery and replication sets *without replacement* (with replacement introduces spurious dependencies)
    X_discovery, X_replication, y_discovery, y_replication = train_test_split(X, y, train_size=sample_size, test_size=sample_size, shuffle=True, random_state=random_state)

    # standard 10-fold cross-validation
    cv = KFold(10)

    # below we obtain cross-validated predictions in the discovery sample
    predicted_discovery_cv = np.zeros_like(y_discovery)  # here we collect the predictions for each fold
    cor_per_fold = np.zeros(cv.n_splits)  # here we collect the predictive performance in each fold
    i = 0  # just a counter
    for train, test in cv.split(X=X_discovery, y=y_discovery):  # loop to leave one fold out
        model.fit(X=X_discovery[train], y=y_discovery[train]) # fit model to the training set
        predicted_discovery_cv[test] = model.predict(X=X_discovery[test]) # use fitted model to predict teh test set
        cor_per_fold[i] = np.corrcoef(y_discovery[test], predicted_discovery_cv[test])[0,1] # calculate performance on tne test set
        i += 1
    # calculate mean test performance across all folds
    r_disc_cv = np.mean(cor_per_fold)
    # 'finalize' model by training it on the full discovery sample (without cross-validation)
    final_model = model.fit(X=X_discovery, y=y_discovery)
    # obtain predictions with the final model on the discovery sample, note that this model actually overfits this sample.
    # we do this only to demonstrate biased estimates
    predicted_discovery_overfit = final_model.predict(X=X_discovery)
    # here we obtain the biased effect size (r) estimates for demonstrational purposes
    r_disc_overfit = np.corrcoef(predicted_discovery_overfit, y_discovery)[0, 1]

    # We use the final model to predict the replication sample
    # This is correct (no overfitting here), the final model did not see this data during training
    predicted_replication = final_model.predict(X=X_replication)
    # we obtain the out-of-sample prediction performance estimates
    r_rep = np.corrcoef(predicted_replication, y_replication)[0, 1]

    # below we calculate permutation-based p-values for all three effect size estimates (in-sample unbiased, in-sample biased, out-of-sample)
    # (one sided tests, testing for positive correlation)
    p_disc_cv = permutation_test(predicted_discovery_cv, y_discovery, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)
    p_disc_overfit = permutation_test(predicted_discovery_overfit, y_discovery, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)
    p_rep = permutation_test(predicted_replication, y_replication, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)
    # return results
    return r_disc_cv, r_disc_overfit, r_rep, p_disc_cv, p_disc_overfit, p_rep

All set, now we start the analysis.

# Replicability with sample sizes n=50, 100, 200, 300 and max
Here we train a few different models on 100 bootstrap samples.

We aggregate the results of our workhorse function in `n_bootstrap`=100 bootstrap cases (run in parallel).

The whole process is repeated for all sample sizes, fetaure_sets and target variables.

## Here we test all 33 variables, proposed by Marek et al. in their response.

In [5]:
targets = ['PicVocab_AgeAdj',
           'Flanker_AgeAdj',
           'ListSort_AgeAdj',
           'CardSort_AgeAdj',
           'ProcSpeed_AgeAdj',
           'PicSeq_AgeAdj',
           'ReadEng_AgeAdj',
           'CogFluidComp_AgeAdj',
           'CogCrystalComp_AgeAdj',
           'CogTotalComp_AgeAdj',
           'PMAT24_A_CR',
           'Sadness_Unadj',
           'FearAffect_Unadj',
           'FearSomat_Unadj',''
           'AngAffect_Unadj',
           'AngAggr_Unadj',
           'AngHostil_Unadj',
           'LifeSatisf_Unadj',
           'MeanPurp_Unadj',
           'PosAffect_Unadj',
           'EmotSupp_Unadj',
           'Friendship_Unadj',
           'PercHostil_Unadj',
           'PercReject_Unadj',
           'PercStress_Unadj',
           'SelfEff_Unadj',
           'InstruSupp_Unadj',
           'Loneliness_Unadj',
           'NEOFAC_O',
           'NEOFAC_C',
           'NEOFAC_E',
           'NEOFAC_A',
           'NEOFAC_N'
           ]

# Reproducing the PCA+SVR-based model from the target paper
### Like in the target paper:
- Both PCA and SVR are done inside the cross-validation
- PCA reatains the firts k principal components that together explain 50% of the variance
- scikit-learn makes sure that PCA is only fit for the training samples
- both for the test sets (in the cross-validation) and the replication sample PCA is not re-fit, bt features are simply transformed with the already fit PCA

In [6]:
%%time

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'PCA_SVR': Pipeline([('pca', PCA(n_components=0.5)),
                         ('svr', SVR())])

}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in targets:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set])

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })
                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print(tmp_data_frame.r_discovery_cv.mean(), tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res/revised_results_PCA_SVR.csv')

df

*****************************************************************
netmats_parcor PCA_SVR PicVocab_AgeAdj 50
0.13188515515855453 0.14770439388426013
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR PicVocab_AgeAdj 100
0.1867148045854563 0.21667608060887125
Replicability at alpha = 0.05 : 75.0 %
Replicability at alpha = 0.01 : 25.0 %
Replicability at alpha = 0.005 : 10.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR PicVocab_AgeAdj 200
0.2677715498715301 0.2931765457541302
Replicability at alpha = 0.05 : 98.59154929577466 %
Replicability at alpha = 0.01 : 78.87323943661971 %
Replicability at alpha = 0.005 : 67.6056338028169 %
Replicability at alpha = 0.001 : 35.2112676056338 %
************************************

<timed exec>:60: RuntimeWarning: invalid value encountered in long_scalars


0.0386050459023509 0.04446606264674254
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR Flanker_AgeAdj 200
0.04725917071923175 0.05833533752802022
Replicability at alpha = 0.05 : 33.33333333333333 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR Flanker_AgeAdj 300
0.07312846509978996 0.07671340073350896
Replicability at alpha = 0.05 : 26.666666666666668 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR Flanker_AgeAdj max
0.11378620577769874 0.12250618953881152
Replicability at alpha = 0

<timed exec>:60: RuntimeWarning: invalid value encountered in long_scalars


0.077594631428072 0.08296991306991304
Replicability at alpha = 0.05 : 33.33333333333333 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR CardSort_AgeAdj 200
0.09773349695774895 0.10256141335871115
Replicability at alpha = 0.05 : 35.294117647058826 %
Replicability at alpha = 0.01 : 5.88235294117647 %
Replicability at alpha = 0.005 : 5.88235294117647 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR CardSort_AgeAdj 300
0.12290527214628566 0.12397123226254751
Replicability at alpha = 0.05 : 55.26315789473685 %
Replicability at alpha = 0.01 : 15.789473684210526 %
Replicability at alpha = 0.005 : 5.263157894736842 %
Replicability at alpha = 0.001 : 5.263157894736842 %
*****************************************************************
netmats_parcor PCA_SVR C

<timed exec>:60: RuntimeWarning: invalid value encountered in long_scalars


KeyboardInterrupt: 

# Now we fit a simple Ridge regression
(no feature selection, no hyperparameter optimization)
This can be expected to perform better on low samples than SVR.

In [8]:
%%time

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'ridge': Ridge()
}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in targets:
            for sample_size in ['max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set])

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })
                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print(tmp_data_frame.r_discovery_cv.mean(), tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res/revised_results_Ridge.csv')

df


*****************************************************************
netmats_parcor ridge PicVocab_AgeAdj max
0.4748017798587243 0.4784423901738578
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 100.0 %
Replicability at alpha = 0.005 : 100.0 %
Replicability at alpha = 0.001 : 100.0 %
*****************************************************************
netmats_parcor ridge Flanker_AgeAdj max
0.13226123494919054 0.14022631939127772
Replicability at alpha = 0.05 : 98.87640449438202 %
Replicability at alpha = 0.01 : 61.79775280898876 %
Replicability at alpha = 0.005 : 42.69662921348314 %
Replicability at alpha = 0.001 : 12.359550561797752 %
*****************************************************************
netmats_parcor ridge ListSort_AgeAdj max
0.27262179944410553 0.2762933878478993
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 100.0 %
Replicability at alpha = 0.005 : 100.0 %
Replicability at alpha = 0.001 : 100.0 %
****************************

KeyError: ['LifeSatif_Unadj']

In [12]:
behavior.columns.values

array(['Release', 'Acquisition', 'Gender', 'Age', '3T_Full_MR_Compl',
       'T1_Count', 'T2_Count', '3T_RS-fMRI_Count', '3T_RS-fMRI_PctCompl',
       '3T_Full_Task_fMRI', '3T_tMRI_PctCompl', 'fMRI_WM_PctCompl',
       'fMRI_Gamb_PctCompl', 'fMRI_Mot_PctCompl', 'fMRI_Lang_PctCompl',
       'fMRI_Soc_PctCompl', 'fMRI_Rel_PctCompl', 'fMRI_Emo_PctCompl',
       '3T_dMRI_Compl', '3T_dMRI_PctCompl', 'dMRI_3T_ReconVrs',
       'fMRI_3T_ReconVrs', '7T_Full_MR_Compl', '7T_RS-fMRI_Count',
       '7T_RS-fMRI_PctCompl', '7T_Full_Task_fMRI', '7T_tMRI_PctCompl',
       'fMRI_Movie_Compl', 'fMRI_Movie_PctCompl', 'fMRI_Ret_Compl',
       'fMRI_Ret_PctCompl', '7T_dMRI_Compl', '7T_dMRI_PctCompl',
       '7T_fMRI_Mov_Vrs', 'MEG_AnyData', 'MEG_FullProt_Compl',
       'MEG_HeadModel_Avail', 'MEG_CortRibn_Avail', 'MEG_Anatomy_Avail',
       'MEG_Anatomy_Compl', 'MEG_Noise_Avail', 'MEG_Noise_Compl',
       'MEG_RS_Avail', 'MEG_RS_Compl', 'MEG_WM_Avail', 'MEG_WM_Compl',
       'MEG_StoryMath_Avail', 'MEG_Sto